In [1]:
import glob
import os
import pandas as pd
# Import your classes (assuming they are in your local path)
import sys
sys.path.append("C:/Nimrod/ai_ds_team/football_quant_package/src")
from quant_football.data.data_loader import DataLoader
from quant_football.features.odds_features import OddsFeatureEngineer
from quant_football.core.config import DataConfig, FeatureConfig, Market
from quant_football.modelling import BayesianPoissonGLMM

2026-01-27 09:34:31,556 - pytensor.configdefaults - WARNING - g++ not available, if using conda: `conda install gxx`
2026-01-27 09:34:31,558 - pytensor.configdefaults - WARNING - g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
2026-01-27 09:34:33,558 - arviz.preview - INFO - arviz_base not installed
2026-01-27 09:34:33,564 - arviz.preview - INFO - arviz_stats not installed
2026-01-27 09:34:33,568 - arviz.preview - INFO - arviz_plots not installed


In [2]:



# 1. Setup paths to your real data
DATA_PATH = "data"  # Change this to your actual folder
all_csv_files = glob.glob(os.path.join(DATA_PATH, "**/*.csv"), recursive=True)

print(f"📂 Found {len(all_csv_files)} CSV files.")

# 2. Initialise your tools with your config
# config = DataConfig() 
data_loader = DataLoader(DataConfig())

# 3. Load, Clean, and Standardise in one go
# This uses the method we refined that concatenates first, then cleans
try:
    combined_df = data_loader.load_dataset(all_csv_files)
    
    print("✅ Data Loaded and Cleaned.")
    print(f"📊 Final Shape: {combined_df.shape}")
    print(f"📅 Date Range: {combined_df['match_date'].min()} to {combined_df['match_date'].max()}")
    
    # 4. Preview to ensure match_date and columns are correct
    display(combined_df.head())

except Exception as e:
    print(f"❌ Ingestion Failed: {e}")

📂 Found 6 CSV files.
2026-01-27 09:34:36,492 - quant_football - INFO - Reading file: data\2020.csv
2026-01-27 09:34:36,589 - quant_football - INFO - Reading file: data\2021.csv
2026-01-27 09:34:36,624 - quant_football - INFO - Reading file: data\2022.csv
2026-01-27 09:34:36,655 - quant_football - INFO - Reading file: data\2023.csv
2026-01-27 09:34:36,688 - quant_football - INFO - Reading file: data\2024.csv
2026-01-27 09:34:36,735 - quant_football - INFO - Reading file: data\2025.csv
['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']
Date
Index(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG',
       ...
       'BMGMCA', 'BVCH', 'BVCD', 'BVCA', 'CLCH', 'CLCD', 'CLCA', 'LBCH',
       'LBCD', 'LBCA'],
      dtype='object', length=163)
HomeTeam
Index(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG',
       ...
       'BMGMCA', 'BVCH', 'BVCD', 'BVCA', 'CLCH', 'CLCD', 'CLCA', 'LBCH',
       'LBCD', 'LBCA']

C:\Nimrod\ai_ds_team\football_quant_package\src\quant_football\data\preprocessor.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['match_date'] = pd.to_datetime(
C:\Nimrod\ai_ds_team\football_quant_package\src\quant_football\data\preprocessor.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = np.nan
C:\Nimrod\ai_ds_team\football_quant_package\src\quant_football\data\preprocessor.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,match_date,home_team_idx,away_team_idx
0,E0,Fulham,Arsenal,0,3,A,0,1,A,C Kavanagh,...,1.89,2.02,1.91,2.13,1.92,2.02,1.87,2020-09-12 12:30:00,9,0
1,E0,Crystal Palace,Southampton,1,0,H,1,0,H,J Moss,...,2.13,1.79,2.17,1.85,2.18,1.79,2.12,2020-09-12 15:00:00,7,21
2,E0,Liverpool,Leeds,4,3,H,3,2,H,M Oliver,...,2.05,1.85,2.08,1.90,2.16,1.84,2.04,2020-09-12 17:30:00,13,11
3,E0,West Ham,Newcastle,0,2,A,0,0,D,S Attwell,...,1.87,2.04,1.88,2.09,1.91,2.02,1.86,2020-09-12 20:00:00,26,17
4,E0,West Brom,Leicester City,0,3,A,0,0,D,A Taylor,...,1.98,1.93,1.99,1.95,2.01,1.91,1.97,2020-09-13 14:00:00,25,12


In [3]:
data_loader.teams_mapping

{'Arsenal': 0,
 'Aston Villa': 1,
 'Bournemouth': 2,
 'Brentford': 3,
 'Brighton': 4,
 'Burnley': 5,
 'Chelsea': 6,
 'Crystal Palace': 7,
 'Everton': 8,
 'Fulham': 9,
 'Ipswich': 10,
 'Leeds': 11,
 'Leicester City': 12,
 'Liverpool': 13,
 'Luton': 14,
 'Man City': 15,
 'Man Utd': 16,
 'Newcastle': 17,
 'Norwich': 18,
 "Nott'm Forest": 19,
 'Sheffield United': 20,
 'Southampton': 21,
 'Sunderland': 22,
 'Tottenham': 23,
 'Watford': 24,
 'West Brom': 25,
 'West Ham': 26,
 'Wolves': 27}

In [4]:
combined_df.head()

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,match_date,home_team_idx,away_team_idx
0,E0,Fulham,Arsenal,0,3,A,0,1,A,C Kavanagh,...,1.89,2.02,1.91,2.13,1.92,2.02,1.87,2020-09-12 12:30:00,9,0
1,E0,Crystal Palace,Southampton,1,0,H,1,0,H,J Moss,...,2.13,1.79,2.17,1.85,2.18,1.79,2.12,2020-09-12 15:00:00,7,21
2,E0,Liverpool,Leeds,4,3,H,3,2,H,M Oliver,...,2.05,1.85,2.08,1.90,2.16,1.84,2.04,2020-09-12 17:30:00,13,11
3,E0,West Ham,Newcastle,0,2,A,0,0,D,S Attwell,...,1.87,2.04,1.88,2.09,1.91,2.02,1.86,2020-09-12 20:00:00,26,17
4,E0,West Brom,Leicester City,0,3,A,0,0,D,A Taylor,...,1.98,1.93,1.99,1.95,2.01,1.91,1.97,2020-09-13 14:00:00,25,12


In [5]:
# Initialise with the package's own classes
engineer = OddsFeatureEngineer(FeatureConfig())

# Copy your data
df = combined_df.copy()

# 1. Run Match Odds Probabilities
df = engineer.calculate_implied_probabilities(df, bookmaker_prefix='B365')

# 2. Run Over/Under Probabilities 
# (Note: This expects columns "B365>2.5" and "B365<2.5" based on your DataConfig)
df = engineer.calculate_implied_probabilities_over_under(df, bookmaker_prefix='B365')

# 3. Get Best Odds
df = engineer.get_best_pre_match_odds(df, Market.MATCH_ODDS)

# Check results
display(df[[c for c in df.columns if 'implied_prob' in c or 'best_odds' in c]].head())

2026-01-27 09:34:37,173 - quant_football - INFO - Calculated implied probabilities for B365 (Sane rows: 2070/2070)
2026-01-27 09:34:37,188 - quant_football - INFO - Calculated O/U implied probabilities for B365 (Sane rows: 0/2070)
2026-01-27 09:34:37,206 - quant_football - INFO - Calculated best pre-match match odds.


,implied_prob_home_B365,implied_prob_draw_B365,implied_prob_away_B365,implied_prob_over_2_5_B365,implied_prob_under_2_5_B365,best_odds_home,best_odds_draw,best_odds_away
0,0.158548,0.219697,0.621756,NaN,NaN,6.55,4.55,1.60
1,0.306573,0.292424,0.401003,NaN,NaN,3.36,3.36,2.50
2,0.741801,0.158251,0.099948,NaN,NaN,1.35,6.50,10.75
3,0.441558,0.279221,0.279221,NaN,NaN,2.24,3.70,3.60
4,0.249733,0.263607,0.486660,NaN,NaN,4.00,3.82,2.04


In [6]:
df[["implied_prob_home_B365", "implied_prob_draw_B365","implied_prob_away_B365"]].sum(axis=1).mean()


1.0

In [7]:
df.shape

(2070, 115)

In [8]:
df_train = df.iloc[:100, :]

In [10]:
# 1. Configuration for testing
# We'll use fewer chains and draws to keep the feedback loop fast
test_config = {
    'sampling': {
        'draws': 500,
        'tune': 500,
        'chains': 2,
        'target_accept': 0.8,
        'random_seed': 42
    },
    'priors': {
        'mu': {'mean': 0.0, 'sd': 0.7},
        'h_adv': {'mean': 0.3, 'sd': 0.1},
        'sigma_att': {'sd': 0.5},
        'sigma_def': {'sd': 0.5},
        'alpha': {'sd': 0.1}
    }
}

teams_mapping = data_loader.teams_mapping
# 4. Fit the model
model = BayesianPoissonGLMM(config=test_config)

trace = model.fit(df_train, teams_mapping=teams_mapping)




2026-01-27 09:45:42,828 - quant_football - INFO - Fitting BayesianPoissonGLMM with 100 matches.
2026-01-27 09:45:43,862 - pymc.sampling.mcmc - INFO - Initializing NUTS using jitter+adapt_diag...
2026-01-27 09:45:47,275 - pymc.sampling.mcmc - INFO - Multiprocess sampling (2 chains in 2 jobs)
2026-01-27 09:45:47,275 - pymc.sampling.mcmc - INFO - NUTS: [mu, h_adv, sigma_att, sigma_def, att, defs, alpha]


Output()

2026-01-27 10:55:39,553 - pymc.sampling.mcmc - INFO - Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 4192 seconds.
2026-01-27 10:55:40,487 - pymc.stats.convergence - INFO - We recommend running at least 4 chains for robust computation of convergence diagnostics
2026-01-27 10:55:40,487 - pymc.stats.convergence - INFO - The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
2026-01-27 10:55:40,487 - pymc.stats.convergence - ERROR - The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [11]:
import arviz as az
summary = az.summary(trace, var_names=["mu", "h_adv", "alpha", "sigma_att", "sigma_def"])
print("\n--- Parameter Summary ---")
print(summary)



--- Parameter Summary ---
            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
mu         0.062  0.122  -0.150    0.300      0.005    0.003     661.0   
h_adv      0.261  0.092   0.092    0.429      0.003    0.003     854.0   
alpha      0.948  0.062   0.827    1.056      0.002    0.002     812.0   
sigma_att  0.184  0.102   0.027    0.369      0.015    0.007      41.0   
sigma_def  0.133  0.102   0.013    0.326      0.029    0.008       7.0   

           ess_tail  r_hat  
mu            573.0   1.00  
h_adv         636.0   1.00  
alpha         692.0   1.00  
sigma_att      81.0   1.01  
sigma_def      31.0   1.22  


In [14]:
# 5. Quick Diagnostic Check

# 6. Test Prediction Logic
# Let's predict a hypothetical match between two teams 

df_test  = df.loc[101:104,['HomeTeam','AwayTeam']].reset_index(drop=True) 
probs = model.predict_outcome_probabilities(df_test)
print("\n--- Prediction for test match ---")
print(pd.concat([df_test, probs], axis=1))

2026-01-27 11:10:41,295 - pymc.sampling.forward - INFO - Sampling: []


Output()

C:\Nimrod\ai_ds_team\football_quant_package\src\quant_football\modelling\bayesian_poisson_glmm.py:153: UserWarning: The effect of Potentials on other parameters is ignored during posterior predictive sampling. This is likely to lead to invalid or biased predictive samples.
  ppc = pm.sample_posterior_predictive(self.trace, var_names=["theta_h_det", "theta_a_det"])



--- Prediction for test match ---
           HomeTeam        AwayTeam  home_win   draw  away_win  over_2_5  \
0           Chelsea           Leeds     0.461  0.274     0.265     0.490   
1         West Brom  Crystal Palace     0.428  0.264     0.308     0.422   
2  Sheffield United  Leicester City     0.399  0.269     0.332     0.448   
3         Tottenham         Arsenal     0.482  0.259     0.259     0.445   

   under_2_5  
0      0.510  
1      0.578  
2      0.552  
3      0.555  
